In [1]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [2]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [3]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [4]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [5]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 4))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 4))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [6]:
readInJson(garbageCollectionDate[4]).head(2)

,type,name,features
0,FeatureCollection,Frankston City Council Garbage Collection Zones,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
1,FeatureCollection,Frankston City Council Garbage Collection Zones,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [7]:
df = readInJson(garbageCollectionDate[4])

df['properties'] = df.features.apply(lambda x: x['properties'])

df['geo'] = df.features.apply(lambda x: x['geometry']['coordinates'][0])

df = df.explode('geo')
df['geo_nested'] = df.geo.apply(lambda x: isNested(x))

nested = df.loc[df.geo_nested == 1].explode('geo')
unnested = df.loc[df.geo_nested == 0]

df = pd.concat([nested, unnested])

df['lat'] = df.geo.apply(lambda x: round(x[0], 4))
df['long'] = df.geo.apply(lambda x: round(x[1], 4))

df.drop(['geo', 'features', 'type', 'name', 'geo_nested'], axis = 1, inplace = True)

In [9]:
df.iloc[0,0]

{'rub_day': 'Tuesday',
 'rub_weeks': 1,
 'rub_start': '20170103',
 'rec_day': 'Tuesday',
 'rec_weeks': 2,
 'rec_start': '20120110',
 'grn_day': 'Tuesday',
 'grn_weeks': 2,
 'grn_start': '20120103',
 'Click for more information': 'https://www.frankston.vic.gov.au/Environment-and-Waste/Waste-and-Recycling/Bin-Information',
 'missed_ph': '1300 322 322'}

In [10]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
    
    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

df.drop('properties', axis = 1, inplace = True)

In [14]:
frankston = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

frankston.to_csv('../GBD_frankston.csv', index = False)